In [27]:
from time import sleep
from tqdm import tqdm

import numpy as np
import pandas as pd
import gspread
from gspread_dataframe import set_with_dataframe
from google.oauth2.service_account import Credentials

from keys import GOOG_API_KEY

In [36]:
## Load dataframes
# Polling averages
pres = pd.read_csv('averages/presidential_gen_approval.csv')
pres_issue = pd.read_csv('averages/presidential_issue_approvals.csv')
pres_rv = pd.read_csv('averages/presidential_RV_approval.csv')

# Polls display tables
pres_polls = pd.read_csv('transformed_tables/gen_approval_polls.csv')
issue_polls = pd.read_csv('transformed_tables/issue_approval_polls.csv')

In [13]:
# Define the scope
scopes = [
    "https://www.googleapis.com/auth/spreadsheets",
    "https://www.googleapis.com/auth/drive"
]

# Authenticate with the service account
credentials = Credentials.from_service_account_file(
    GOOG_API_KEY,
    scopes=scopes
)

# Authorize gspread
client = gspread.authorize(credentials)

## Polling Averages

In [21]:
spreadsheet = client.open("snoutcounter-data")
worksheet = spreadsheet.worksheet("approval-stats")
worksheet.clear()

set_with_dataframe(worksheet, pres.reset_index())

In [22]:
worksheet = spreadsheet.worksheet("approval_stats_RV")
worksheet.clear()

set_with_dataframe(worksheet, pres_rv.reset_index())

In [25]:
issues = ['economy', 'immigration', 'inflation', 'trade_tariffs', 'foreign_policy', 'healthcare', 'crime']
issue_suffixes = ['econ', 'imm', 'infl', 'trade', 'fp', 'healthcare', 'crime']

for iss, suf in tqdm(zip(issues, issue_suffixes)):
    worksheet = spreadsheet.worksheet(f"approval_stats_{suf}")
    worksheet.clear()

    issue_df = pres_issue[pres_issue['issue'] == iss]
    set_with_dataframe(worksheet, issue_df.reset_index())

7it [00:05,  1.23it/s]


In [32]:
## Net issue approval
net_issues = pd.pivot_table(pres_issue[pres_issue['issue'].isin(issues)], 
                            values=['net', 'net_lower_ci', 'net_upper_ci'], index=['end_date'], columns=['issue'],
                           aggfunc="first").reset_index()
net_issues.columns = ['_'.join(col).strip() for col in net_issues.columns.values]
net_issues.head()

,end_date_,net_crime,net_economy,net_foreign_policy,net_healthcare,net_immigration,net_inflation,net_trade_tariffs,net_lower_ci_crime,net_lower_ci_economy,...,net_lower_ci_immigration,net_lower_ci_inflation,net_lower_ci_trade_tariffs,net_upper_ci_crime,net_upper_ci_economy,net_upper_ci_foreign_policy,net_upper_ci_healthcare,net_upper_ci_immigration,net_upper_ci_inflation,net_upper_ci_trade_tariffs
0,2025-01-21,0.0,7.918073,3.918073,0.000000,9.918073,-4.081927,-0.081927,0.0,7.918073,...,9.918073,-4.081927,-0.081927,0.0,7.918073,3.918073,0.000000,9.918073,-4.081927,-0.081927
1,2025-01-22,0.0,7.918073,3.918073,0.000000,9.918073,-4.081927,-0.081927,0.0,7.918073,...,9.918073,-4.081927,-0.081927,0.0,7.918073,3.918073,0.000000,9.918073,-4.081927,-0.081927
2,2025-01-23,0.0,2.633739,3.918073,-13.485259,3.206184,-4.081927,-0.081927,0.0,-3.924942,...,-5.124316,-4.081927,-0.081927,0.0,9.192420,3.918073,-13.485259,11.536684,-4.081927,-0.081927
3,2025-01-24,0.0,2.633739,3.918073,-13.485259,3.206184,-4.081927,-0.081927,0.0,-3.924942,...,-5.124316,-4.081927,-0.081927,0.0,9.192420,3.918073,-13.485259,11.536684,-4.081927,-0.081927
4,2025-01-25,0.0,2.633739,3.918073,-13.485259,3.206184,-4.081927,-0.081927,0.0,-3.924942,...,-5.124316,-4.081927,-0.081927,0.0,9.192420,3.918073,-13.485259,11.536684,-4.081927,-0.081927


In [34]:
worksheet = spreadsheet.worksheet("net_approval_issues")
worksheet.clear()

set_with_dataframe(worksheet, net_issues.reset_index())

## Polls Display Tables

In [38]:
pres_polls['sponsors'] = '^Sponsor: ' + pres_polls['sponsors'] + '^'
pres_polls['sponsors'] = pres_polls['sponsors'].fillna('')
pres_polls = pres_polls.sort_values(['total_weight'], ascending=False)
pres_polls = pres_polls.rename({
    'pollster':'Pollster', 'approve':'Approve', 'disapprove':'Disapprove', 'net':'Net', 
    'total_weight':'Weight', 'start_date':'Start Date', 'end_date':'End Date', 'url':'URL', 'adj_net':'Adjusted Net'}, axis='columns')
pres_polls['Sample'] = pres_polls['sample_size'].astype(int).astype(str) + ' ' + pres_polls['population']
pres_polls['Start Date'] = pd.to_datetime(pres_polls['Start Date']).astype(str)
pres_polls['End Date'] = pres_polls['End Date'].astype(str)
pres_polls = pres_polls.reset_index()
pres_polls['Pollster'] = ('<a href="' + pres_polls['URL'] + '" style="color:black; text-decoration:underline;"><u>' + 
                          pres_polls['Pollster'] + '</u></a>' + pres_polls['sponsors'])
pres_polls = pres_polls[['Pollster', 'Start Date', 'End Date', 'Sample', 'Weight', 'Approve', 'Disapprove', 'Net', 'Adjusted Net']]
pres_polls

,Pollster,Start Date,End Date,Sample,Weight,Approve,Disapprove,Net,Adjusted Net
0,"<a href=""https://d3nkl3psvxxpe9.cloudfront.net...",2025-12-26,2025-12-29,1550 A,9.791500e-02,39.00,56.00,-17.00,-15.492450
1,"<a href=""https://www.atlasintel.org/poll/usa-n...",2025-12-15,2025-12-19,2315 A,9.202994e-02,39.30,59.60,-20.30,-20.871479
2,"<a href=""https://www.cbsnews.com/news/opinion-...",2025-12-17,2025-12-19,2300 A,6.901086e-02,41.00,59.00,-18.00,-16.492450
3,"<a href=""https://www.dailymail.co.uk/news/arti...",2025-12-20,2025-12-21,1000 RV,5.312326e-02,43.36,46.78,-3.42,-10.846362
4,"<a href=""https://independentcentervoice.com/in...",2025-12-30,2026-01-01,1200 A,4.996633e-02,38.00,58.00,-20.00,-20.303512
...,...,...,...,...,...,...,...,...,...
510,"<a href=""https://www.reuters.com/world/us/amer...",2025-01-24,2025-01-26,1034 A,1.927150e-18,45.00,46.00,-1.00,3.054713
511,"<a href=""https://pro.morningconsult.com/tracke...",2025-01-24,2025-01-26,2302 RV,1.820373e-18,52.00,44.00,8.00,3.703138
512,"<a href=""https://substack.com/inbox/post/15539...",2025-01-21,2025-01-21,742 A,1.464594e-18,49.00,36.00,13.00,8.201816
513,"<a href=""https://www.ipsos.com/en-us/reuters-i...",2025-01-20,2025-01-21,1077 A,1.136757e-18,47.00,41.00,6.00,10.054713


In [39]:
worksheet = spreadsheet.worksheet("polls_display_table")
worksheet.clear()

set_with_dataframe(worksheet, pres_polls)